In [1]:
from langchain_community.document_loaders import CSVLoader
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter

csv_load = CSVLoader('netflix_titles_1000.csv', metadata_columns=['show_id','date_added','release_year'])
docs = csv_load.load()
splitter = RecursiveCharacterTextSplitter(separators=["\n",","," ",""],chunk_size=70,chunk_overlap=10)

chunks = splitter.split_documents(docs)

c:\Users\gaura\anaconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv,find_dotenv
import os
env_path = find_dotenv(filename='password.env',usecwd=True)
load_dotenv(env_path)
key = os.getenv('OPENAI_API_KEY')
#key

In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')
vector_store = Chroma.from_documents(documents=chunks, embedding = embedding_model)



In [4]:
retriever = vector_store.as_retriever(search_type='similarity',
                                      search_kwargs={'k':3})

In [5]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model = 'gpt-4o-mini', api_key=key )

prompt = ChatPromptTemplate.from_template("""You are an expert movie reccomendation engine.
                                          Based on the context, answer the question. The context is. 
                                          If the question is unrelated to movies say you do not know
                                          context: {context}
                                          question: {question}""")
chain = ({'context':retriever,'question':RunnablePassthrough()}
         | prompt
         | llm
         |StrOutputParser())

result = chain.invoke("""question: I love action movies. 
                      Recommend 2 action movies and in 
                      1 line explain the reason for the reccomendation""")

In [6]:
print(result)

1. **The Wrestler (2008)** - This film combines intense action with a compelling character study in the world of professional wrestling, highlighting the struggles and triumphs within the sport.

2. **Warrior (2011)** - A gripping drama that revolves around mixed martial arts, showcasing high-stakes action and deep emotional ties among the characters competing in the ring.


In [9]:
result=chain.invoke("""question: I liked Shawshank redemption. 
                      Recommend 2 movies similar to Shawshank Redemption and in 
                      1 line explain the reason for the reccomendation and similarities to Shawshank redemption""")

In [10]:
print(result)

I recommend "Mystic River" and "The Green Mile." 

1. **Mystic River** - Directed by Clint Eastwood, like Shawshank Redemption, it explores deep themes of friendship, loss, and the impact of the past on present lives. 
2. **The Green Mile** - This film, based on a Stephen King story, shares a profound narrative about justice, morality, and the human spirit, resonating with the emotional depth found in Shawshank Redemption.


In [59]:
import pandas as pd
data = pd.read_csv('netflix_titles_1000.csv')
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [60]:
from langchain_core.documents import Document
docs2 = []
for _,row in data.iterrows():
    text = f"""  title:{row['title']} \n listed_in:{row['listed_in']}\n description:{row['description']} """
    docs2.append(Document(page_content=text,metadata={'show_id':row['show_id']}))

In [61]:
print(docs2[0].page_content)

  title:Dick Johnson Is Dead 
 listed_in:Documentaries
 description:As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable. 


In [63]:
vector_store = Chroma.from_documents(documents=docs2,
                                     embedding = embedding_model,
                                     persist_directory="chroma_netflix_v2")

In [73]:
query_vector = embedding_model.embed_query('I love international action movies')

In [74]:
results = vector_store.similarity_search_by_vector(query_vector,k=3)

In [75]:
texts = [doc.page_content for doc in results]

In [76]:
for i, t in enumerate(texts):
    print(f"\n--- Result {i} ---")
    print(t)


--- Result 0 ---
  title:Xtreme 
 listed_in:Action & Adventure, International Movies
 description:In this fast-paced and action-packed thriller, a retired hitman — along with his sister and a troubled teen — takes revenge on his lethal stepbrother. 

--- Result 1 ---
  title:Jagame Thandhiram 
 listed_in:Action & Adventure, Dramas, International Movies
 description:When a clever, carefree gangster is recruited to help an overseas crime lord take down a rival, he is caught off guard by the moral dilemmas that follow. 

--- Result 2 ---
  title:Department 
 listed_in:Action & Adventure, International Movies
 description:Two cops form a task force to take down two mobsters, but when a sinister politician enters the picture, dubious loyalties and motives come to light. 
